In [ ]:
from fastai.gen_doc.gen_notebooks import update_nb_metadata
# For updating jekyll metadata. You MUST reload notebook immediately after executing this cell for changes to save
update_nb_metadata('models.rnn.ipynb', title='models.rnn', summary=None)

# models.rnn

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.models.rnn import * 

This module fully implements the [AWD-LSTM](https://arxiv.org/pdf/1708.02182.pdf) from Stephen Merity et al. The main idea of the article is to use a [RNN](http://www.pnas.org/content/79/8/2554) with dropout everywhere, but in an intelligent way. There is a difference with the usual dropout, which is why you’ll see a `RNNDropout` module: we zero things, as is usual in dropout, but we always zero the same thing according to the sequence dimension (which is the first dimension in pytorch). This ensures consistency when updating the hidden state through the whole sentences/articles. 

This being given, there are five different dropouts in the AWD-LSTM:
- the first one, embedding dropout, is applied when we look the ids of our tokens inside the embedding matrix (to transform them from numbers to a vector of float). We zero some lines of it, so random ids are sent to a vector of zeros instead of being sent to their embedding vector.
- the second one, input dropout, is applied to the result of the embedding with dropout. We forget random pieces of the embedding matrix (but as stated in the last paragraph, the same ones in the sequence dimension).
- the third one is the weight dropout. It’s the trickiest to implement as we randomly replace by 0s some weights of the hidden-to-hidden matrix inside the RNN: this needs to be done in a way that ensure the gradients are still computed and the initial weights still updated.
- the fourth one is the hidden dropout. It’s applied to the output of one of the layers of the RNN before it’s used as input of the next layer (again same coordinates are zeroed in the sequence dimension). This one isn’t applied to the last output, but rather…
- the fifth one is the output dropout, it’s applied to the last output of the model (and like the others, it’s applied the same way through the first dimension).

## Basic functions to get a model

In [ ]:
show_doc(get_language_model, doc_string=False)

#### <a id=get_language_model></a>`get_language_model`
> `get_language_model`(`vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `tie_weights`:`bool`=`True`, `qrnn`:`bool`=`False`, `bias`:`bool`=`True`, `bidir`:`bool`=`False`, `output_p`:`float`=`0.4`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L205">[source]</a>

Creates an AWD-LSTM with a first embedding of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, RNNs with `n_layers` that can be bidirectional if `bidir` is True. The last RNN as an output size of `emb_sz` so that we can use the same decoder as the encoder if `tie_weights` is True. The decoder is a `Linear` layer with or without `bias`. If `qrnn` is set to True, we use [QRNN cells] instead of LSTMS. `pad_token` is the token used for padding.

`embed_p` is used for the embedding dropout, `input_p` is used for the input dropout, `weight_p` is used for the weight dropout, `hidden_p` is used for the hidden dropout and `output_p` is used for the output dropout.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the `RNNTrainer`). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using `RNNTrainer`.

In [ ]:
show_doc(get_rnn_classifier, doc_string=False)

#### <a id=get_rnn_classifier></a>`get_rnn_classifier`
> `get_rnn_classifier`(`bptt`:`int`, `max_seq`:`int`, `n_class`:`int`, `vocab_sz`:`int`, `emb_sz`:`int`, `n_hid`:`int`, `n_layers`:`int`, `pad_token`:`int`, `layers`:`Collection`\[`int`\], `drops`:`Collection`\[`float`\], `bidir`:`bool`=`False`, `qrnn`:`bool`=`False`, `hidden_p`:`float`=`0.2`, `input_p`:`float`=`0.6`, `embed_p`:`float`=`0.1`, `weight_p`:`float`=`0.5`) -> [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)
<a href="https://github.com/fastai/fastai/blob/master/fastai/models/rnn.py#L214">[source]</a>

Creates a RNN classifier with a encoder taken from an AWD-LSTM with arguments `vocab_sz`, `emb_sz`, `n_hid`, `n_layers`, `bias`, `bidir`, `qrnn`, `pad_token` and the dropouts parameters. This encoder is fed the sequence by successive bits of size `bptt` and we only keep the last `max_seq` outputs for the pooling layers.

The decoder use a concatenation of the last outputs, a `MaxPooling` of all the ouputs and an `AveragePooling` of all the outputs. It then uses a list of `BatchNorm`, `Dropout`, `Linear`, `ReLU` blocks (with no `ReLU` in the last one), using a first layer size of `3*emb_sz` then follwoing the numbers in `n_layers` to stop at `n_class`. The dropouts probabilities are read in `drops`.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the `RNNTrainer`). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using `RNNTrainer`.

### Global Variable Definitions:

In [ ]:
show_doc(EmbeddingDropout)

### <a id=EmbeddingDropout></a><em>class</em> `EmbeddingDropout`
(<code>emb</code>:[<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), <code>embed_p</code>:<code>float</code>) :: Inherits ([<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L50">[source]</a></div>


Applies dropout in the embedding layer by zeroing out some elements of the embedding vector.

[`EmbeddingDropout`](/models.rnn.html#EmbeddingDropout)

In [ ]:
show_doc(EmbeddingDropout.forward)

#### <a id=forward></a>`forward`
(<code>words</code>:<code>LongTensor</code>, <code>scale</code>:`Optional`[<code>float</code>]=`None`) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L59">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`EmbeddingDropout.forward`

In [ ]:
show_doc(LinearDecoder)

### <a id=LinearDecoder></a><em>class</em> `LinearDecoder`
(<code>n_out</code>:<code>int</code>, <code>n_hid</code>:<code>int</code>, <code>output_p</code>:<code>float</code>, <code>tie_encoder</code>:[<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=`None`, <code>bias</code>:<code>bool</code>=`True`) :: Inherits ([<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L132">[source]</a></div>


To go on top of a RNN_Core module

[`LinearDecoder`](/models.rnn.html#LinearDecoder)

In [ ]:
show_doc(LinearDecoder.forward)

#### <a id=forward></a>`forward`
(<code>input</code>:<code>Tuple</code>[<code>Tensor</code>, <code>Tensor</code>]) -> <code>Tuple</code>[<code>Tensor</code>, <code>Tensor</code>, <code>Tensor</code>]<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L145">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`LinearDecoder.forward`

In [ ]:
show_doc(MultiBatchRNNCore)

### <a id=MultiBatchRNNCore></a><em>class</em> `MultiBatchRNNCore`
(<code>bptt</code>:<code>int</code>, <code>max_seq</code>:<code>int</code>, <code>args</code>, <code>kwargs</code>) :: Inherits ([<code>RNNCore</code>](fastai.models.rnn.html#RNNCore))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L157">[source]</a></div>


Creates a RNNCore module that can process a full sentence.

[`MultiBatchRNNCore`](/models.rnn.html#MultiBatchRNNCore)

In [ ]:
show_doc(MultiBatchRNNCore.concat)

#### <a id=concat></a>`concat`
(<code>arrs</code>:<code>Collection</code>[<code>Tensor</code>]) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L164">[source]</a></div>


Concatenates the arrays along the batch dimension.

`MultiBatchRNNCore.concat`

In [ ]:
show_doc(MultiBatchRNNCore.forward)

#### <a id=forward></a>`forward`
(<code>input</code>:<code>LongTensor</code>) -> <code>Tuple</code>[<code>Tensor</code>, <code>Tensor</code>]<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L168">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`MultiBatchRNNCore.forward`

In [ ]:
show_doc(PoolingLinearClassifier)

### <a id=PoolingLinearClassifier></a><em>class</em> `PoolingLinearClassifier`
(<code>layers</code>:<code>Collection</code>[<code>int</code>], <code>drops</code>:<code>Collection</code>[<code>float</code>]) :: Inherits ([<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L179">[source]</a></div>


Creates a linear classifier with pooling.

[`PoolingLinearClassifier`](/models.rnn.html#PoolingLinearClassifier)

In [ ]:
show_doc(PoolingLinearClassifier.forward)

#### <a id=forward></a>`forward`
(<code>input</code>:<code>Tuple</code>[<code>Tensor</code>, <code>Tensor</code>]) -> <code>Tuple</code>[<code>Tensor</code>, <code>Tensor</code>, <code>Tensor</code>]<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L195">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`PoolingLinearClassifier.forward`

In [ ]:
show_doc(PoolingLinearClassifier.pool)

#### <a id=pool></a>`pool`
(<code>x</code>:<code>Tensor</code>, <code>bs</code>:<code>int</code>, <code>is_max</code>:<code>bool</code>)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L190">[source]</a></div>


Pools the tensor along the seq_len dimension.

`PoolingLinearClassifier.pool`

In [ ]:
show_doc(RNNCore)

### <a id=RNNCore></a><em>class</em> `RNNCore`
(<code>vocab_sz</code>:<code>int</code>, <code>emb_sz</code>:<code>int</code>, <code>n_hid</code>:<code>int</code>, <code>n_layers</code>:<code>int</code>, <code>pad_token</code>:<code>int</code>, <code>bidir</code>:<code>bool</code>=`False`, <code>hidden_p</code>:<code>float</code>=`0.2`, <code>input_p</code>:<code>float</code>=`0.6`, <code>embed_p</code>:<code>float</code>=`0.1`, <code>weight_p</code>:<code>float</code>=`0.5`, <code>qrnn</code>:<code>bool</code>=`False`) :: Inherits ([<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L73">[source]</a></div>


AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182

[`RNNCore`](/models.rnn.html#RNNCore)

In [ ]:
show_doc(RNNCore.forward)

#### <a id=forward></a>`forward`
(<code>input</code>:<code>LongTensor</code>) -> <code>Tuple</code>[<code>Tensor</code>, <code>Tensor</code>]<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L104">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`RNNCore.forward`

In [ ]:
show_doc(RNNCore.one_hidden)

#### <a id=one_hidden></a>`one_hidden`(<code>l</code>:<code>int</code>) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L120">[source]</a></div>


Returns one hidden state

`RNNCore.one_hidden`

In [ ]:
show_doc(RNNCore.reset)

#### <a id=reset></a>`reset`()<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L125">[source]</a></div>


Resets the hidden states

`RNNCore.reset`

In [ ]:
show_doc(RNNDropout)

### <a id=RNNDropout></a><em>class</em> `RNNDropout`(<code>p</code>:<code>float</code>=`0.5`) :: Inherits ([<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L11">[source]</a></div>


Dropout that is consistent on the seq_len dimension

[`RNNDropout`](/models.rnn.html#RNNDropout)

In [ ]:
show_doc(RNNDropout.forward)

#### <a id=forward></a>`forward`(<code>x</code>:<code>Tensor</code>) -> <code>Tensor</code><div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L18">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`RNNDropout.forward`

In [ ]:
show_doc(SequentialRNN)

### <a id=SequentialRNN></a><em>class</em> `SequentialRNN`(<code>args</code>) :: Inherits ([<code>Sequential</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L151">[source]</a></div>


A sequential module that passes the reset call to its children.

[`SequentialRNN`](/models.rnn.html#SequentialRNN)

In [ ]:
show_doc(SequentialRNN.reset)

#### <a id=reset></a>`reset`()<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L153">[source]</a></div>

`SequentialRNN.reset`

In [ ]:
show_doc(WeightDropout)

### <a id=WeightDropout></a><em>class</em> `WeightDropout`
(<code>module</code>:[<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), <code>weight_p</code>:<code>float</code>, <code>layer_names</code>:<code>Collection</code>[<code>str</code>]=`['weight_hh_l0']`) :: Inherits ([<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module))<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L23">[source]</a></div>


A module that warps another layer in which some weights will be replaced by 0 during training.

[`WeightDropout`](/models.rnn.html#WeightDropout)

In [ ]:
show_doc(WeightDropout._setweights)

#### <a id=_setweights></a>`_setweights`()<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L34">[source]</a></div>


Applies dropout to the raw weights

`WeightDropout._setweights`

In [ ]:
show_doc(WeightDropout.forward)

#### <a id=forward></a>`forward`(<code>args</code>:<code>Collection</code>[<code>Any</code>])<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L40">[source]</a></div>


Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.

`WeightDropout.forward`

In [ ]:
show_doc(WeightDropout.reset)

#### <a id=reset></a>`reset`()<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L47">[source]</a></div>

`WeightDropout.reset`

In [ ]:
show_doc(dropout_mask)

#### <a id=dropout_mask></a>`dropout_mask`
(<code>x</code>:<code>Tensor</code>, <code>sz</code>:<code>Collection</code>[<code>int</code>], <code>p</code>:<code>float</code>)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L7">[source]</a></div>


Returns a dropout mask of the same type as x, size sz, with probability p to cancel an element.

[`dropout_mask`](/models.rnn.html#dropout_mask)

In [ ]:
show_doc(get_language_model)

#### <a id=get_language_model></a>`get_language_model`
(<code>vocab_sz</code>:<code>int</code>, <code>emb_sz</code>:<code>int</code>, <code>n_hid</code>:<code>int</code>, <code>n_layers</code>:<code>int</code>, <code>pad_token</code>:<code>int</code>, <code>tie_weights</code>:<code>bool</code>=`True`, <code>qrnn</code>:<code>bool</code>=`False`, <code>bias</code>:<code>bool</code>=`True`, <code>output_p</code>:<code>float</code>=`0.4`, <code>hidden_p</code>:<code>float</code>=`0.2`, <code>input_p</code>:<code>float</code>=`0.6`, <code>embed_p</code>:<code>float</code>=`0.1`, <code>weight_p</code>:<code>float</code>=`0.5`) -> [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L205">[source]</a></div>


To create a full AWD-LSTM

[`get_language_model`](/models.rnn.html#get_language_model)

In [ ]:
show_doc(get_rnn_classifier)

#### <a id=get_rnn_classifier></a>`get_rnn_classifier`
(<code>bptt</code>:<code>int</code>, <code>max_seq</code>:<code>int</code>, <code>n_class</code>:<code>int</code>, <code>vocab_sz</code>:<code>int</code>, <code>emb_sz</code>:<code>int</code>, <code>n_hid</code>:<code>int</code>, <code>n_layers</code>:<code>int</code>, <code>pad_token</code>:<code>int</code>, <code>layers</code>:<code>Collection</code>[<code>int</code>], <code>drops</code>:<code>Collection</code>[<code>float</code>], <code>bidir</code>:<code>bool</code>=`False`, <code>qrnn</code>:<code>bool</code>=`False`, <code>hidden_p</code>:<code>float</code>=`0.2`, <code>input_p</code>:<code>float</code>=`0.6`, <code>embed_p</code>:<code>float</code>=`0.1`, <code>weight_p</code>:<code>float</code>=`0.5`) -> [<code>Module</code>](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L214">[source]</a></div>


Creates a RNN classifier model

[`get_rnn_classifier`](/models.rnn.html#get_rnn_classifier)

In [ ]:
show_doc(repackage_var)

#### <a id=repackage_var></a>`repackage_var`
(<code>h</code>:<code>None</code>[<code>Tensor</code>, <code>Collection</code>[<code>str</code>]]) -> <code>None</code>[<code>Tensor</code>, <code>Collection</code>[<code>str</code>]]<div style="text-align: right"><a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/models/rnn.py#L69">[source]</a></div>


Detaches h from its history.

[`repackage_var`](/models.rnn.html#repackage_var)